In [17]:
import spacy
from spacy.tokens import DocBin

from spacy.training import Example, offsets_to_biluo_tags
from spacy.training.corpus import Corpus
from spacy.training.loop import train
from spacy.util import load_config

from spacy import displacy

from sklearn.model_selection import train_test_split
import json

from spacy.scorer import Scorer

import random

In [18]:
# Função para converter JSON anotado
def convert_json_to_spacy_format(json_data):
    spacy_format_data = []

    for entry in json_data:
        text = entry['text']
        entities = entry['label']
        spacy_entities = []
        
        # Convertendo cada entidade para o formato spacy
        for entity in entities:
            start, end, label = entity
            spacy_entities.append((start, end, label))
        
        # Adicionando ao formato do spaCy
        spacy_format_data.append((text,spacy_entities))
    
    return spacy_format_data

In [19]:
with open('data\\corpus_annotated\\henrique.jsonl', 'r', encoding='utf-8') as f:
    json_data = [json.loads(line) for line in f]

json_data

[{'id': 101,
  'text': "O jogo desta noite/madrugada que decide o Aberto da Austrália pode ser monótono e aborrecido. Basta Pete Sampras ratificar sua condição de melhor tenista do mundo e destruir os sonhos de seu compatriota Todd Martin em três tranquilos sets. Mas há chance de que a lógica abandone o Flinders Park. Martin dá sinais de que pode virar herói em Melbourne.\nSinais tímidos, é verdade, mas suficientes para a especulação. Nos últimos dois jogos, Martin venceu cinco tie-breaks, dois contra MaliVai Washington e três contra Stefan Edberg. No piso rápido utilizado no torneio, é prova incontestável de que está sacando muito bem.\nSampras chegou perto da perfeição ao arrasar Jim Courier nas semifinais em três sets, 6/3, 6/4 e 6/4. Courier tentava o terceiro título na Austrália e jogou bem, mas foi totalmente dominado. Para superar Sampras, Martin vai ter que mostrar ainda mais do que exibiu ao vencer Edberg por 3/6, 7/6 (9/7), 7/6 (9/7) e 7/6 (7/4).\nAos 23 anos, Martin começa b

In [20]:
# Convertendo o JSON para Spacy
data = convert_json_to_spacy_format(json_data)

data

[("O jogo desta noite/madrugada que decide o Aberto da Austrália pode ser monótono e aborrecido. Basta Pete Sampras ratificar sua condição de melhor tenista do mundo e destruir os sonhos de seu compatriota Todd Martin em três tranquilos sets. Mas há chance de que a lógica abandone o Flinders Park. Martin dá sinais de que pode virar herói em Melbourne.\nSinais tímidos, é verdade, mas suficientes para a especulação. Nos últimos dois jogos, Martin venceu cinco tie-breaks, dois contra MaliVai Washington e três contra Stefan Edberg. No piso rápido utilizado no torneio, é prova incontestável de que está sacando muito bem.\nSampras chegou perto da perfeição ao arrasar Jim Courier nas semifinais em três sets, 6/3, 6/4 e 6/4. Courier tentava o terceiro título na Austrália e jogou bem, mas foi totalmente dominado. Para superar Sampras, Martin vai ter que mostrar ainda mais do que exibiu ao vencer Edberg por 3/6, 7/6 (9/7), 7/6 (9/7) e 7/6 (7/4).\nAos 23 anos, Martin começa bem a temporada depois

In [21]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.125, random_state=42)

In [22]:
train_data

[('A temporada no futebol deste ano prevê uma disputa incomum. Depois do confronto de estruturas que envolveu Palmeiras e São Paulo, no ano passado, os dois clubes voltarão a duelar fora de campo: os técnicos Telê Santana e Wanderley Luxemburgo querem encerrar a temporada em uma posição privilegiada. E, apesar da forte concorrência de Carlos Alberto Parreira (que estará na Copa do Mundo), os dois treinadores ambicionam o título de o mais prestigiado do futebol brasileiro.\nA disputa foi detonada em agosto de 93, quando Telê e Luxemburgo abandonaram a falsidade –um pequeno incidente em um torneio disputado em Cádiz, na Espanha, detonou a rivalidade velada. Irritado com o que viu como violência adversária, o técnico são-paulino dispensou a diplomacia: ao ver a mão estendida de Luxemburgo, após o jogo, respondeu com um aceno para ir embora.\nO tiroteio continuou depois, no vestiário. "Eles ganharam porque o esquema Parmalat funcionou novamente", disparou Telê, insinuando influência da arb

In [23]:
print(f'Tamanho do treino: {len(train_data)}')
print(f'Tamanho da validação: {len(val_data)}')
print(f'Tamanho do teste: {len(test_data)}')

Tamanho do treino: 69
Tamanho da validação: 10
Tamanho do teste: 20


In [24]:
nlp = spacy.blank("pt")  # Modelo vazio em português
ner = nlp.add_pipe("ner")


In [25]:
"""# Adicionar os rótulos das entidades
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])"""

'# Adicionar os rótulos das entidades\nfor _, annotations in train_data:\n    for ent in annotations.get("entities"):\n        ner.add_label(ent[2])'

##### Treinamento do modelo

In [26]:
optimizer = nlp.initialize()

for itn in range(100):  # Número de épocas
    random.shuffle(train_data)
    print(f'('+str(itn)+') Época')
    for raw_text, entity_offsets in train_data:
        # Atualizando o modelo com os textos e entidades
        doc = nlp.make_doc(raw_text)
        entities = {"entities": entity_offsets}
        example =  Example.from_dict(doc, entities)
        nlp.update([example], sgd=optimizer)

(0) Época


c:\Users\Flor\OneDrive - PUCRS - BR\7 Semestre 2024-2\7 Introdução ao Processamento da Linguagem Natural\T1\code\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "O time de juniores do Corinthians é um dos mais ba..." with entities "[(22, 33, 'time'), (66, 84, 'campeonato'), (235, 2...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Flor\OneDrive - PUCRS - BR\7 Semestre 2024-2\7 Introdução ao Processamento da Linguagem Natural\T1\code\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "O título de revelação da Copa São Paulo de Futebol..." with entities "[(25, 57, 'campeonato'), (193, 211, 'time'), (353,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Mi

(1) Época
(2) Época
(3) Época
(4) Época
(5) Época
(6) Época
(7) Época
(8) Época
(9) Época
(10) Época
(11) Época
(12) Época
(13) Época
(14) Época
(15) Época
(16) Época
(17) Época
(18) Época
(19) Época
(20) Época
(21) Época
(22) Época
(23) Época
(24) Época
(25) Época
(26) Época
(27) Época
(28) Época
(29) Época
(30) Época
(31) Época
(32) Época
(33) Época
(34) Época
(35) Época
(36) Época
(37) Época
(38) Época
(39) Época
(40) Época
(41) Época
(42) Época
(43) Época
(44) Época
(45) Época
(46) Época
(47) Época
(48) Época
(49) Época
(50) Época
(51) Época
(52) Época
(53) Época
(54) Época
(55) Época
(56) Época
(57) Época
(58) Época
(59) Época
(60) Época
(61) Época
(62) Época
(63) Época
(64) Época
(65) Época
(66) Época
(67) Época
(68) Época
(69) Época
(70) Época
(71) Época
(72) Época
(73) Época
(74) Época
(75) Época
(76) Época
(77) Época
(78) Época
(79) Época
(80) Época
(81) Época
(82) Época
(83) Época
(84) Época
(85) Época
(86) Época
(87) Época
(88) Época
(89) Época
(90) Época
(91) Época
(92) Épo

##### Avaliação do modelo

In [27]:
examples = []
for raw_text, entity_offsets in test_data:
    doc = nlp.make_doc(raw_text)
    entities = {"entities": entity_offsets}
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [28]:
# Avaliação do modelo
scores = nlp.evaluate(examples)

In [29]:
# Acessar a precisão geral
print("Precisão geral:", scores["ents_p"])
print("Recall:", scores["ents_r"])
print("F1 Score:", scores["ents_f"])

# Acessar métricas específicas para cada tipo de entidade
for ent_type, metrics in scores["ents_per_type"].items():
    print(f"\nEntidade: {ent_type}")
    print(f" - Precision: {metrics['p']}")
    print(f" - Recall: {metrics['r']}")
    print(f" - F1 Score: {metrics['f']}")


Precisão geral: 0.7155963302752294
Recall: 0.5977011494252874
F1 Score: 0.6513569937369521

Entidade: time
 - Precision: 0.8363636363636363
 - Recall: 0.7931034482758621
 - F1 Score: 0.8141592920353982

Entidade: placar
 - Precision: 1.0
 - Recall: 0.5769230769230769
 - F1 Score: 0.7317073170731707

Entidade: estádio
 - Precision: 0.5384615384615384
 - Recall: 0.875
 - F1 Score: 0.6666666666666667

Entidade: campeonato
 - Precision: 0.7647058823529411
 - Recall: 0.65
 - F1 Score: 0.7027027027027027

Entidade: goleiro
 - Precision: 0.6666666666666666
 - Recall: 0.5
 - F1 Score: 0.5714285714285715

Entidade: técnico
 - Precision: 0.6923076923076923
 - Recall: 0.2903225806451613
 - F1 Score: 0.4090909090909091

Entidade: atacante
 - Precision: 0.35714285714285715
 - Recall: 0.38461538461538464
 - F1 Score: 0.3703703703703704

Entidade: defensor
 - Precision: 0.4
 - Recall: 0.4
 - F1 Score: 0.4000000000000001

Entidade: meio-campista
 - Precision: 0.25
 - Recall: 0.17647058823529413
 - F1 

In [30]:
text = "O Inter perdeu de 2 a 0 na Copa Libertadores para o Grêmio. O treinador foi o Celso Roth. No estádio Beira Rio."
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)